论文《Distributed Representations of Words and Phrases and their Compositionality》：https://proceedings.neurips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf

# 词向量基础

计算机中如何表示一个词：“John likes to watch movies. Mary likes too.”

计算机表示会生成一个词典：{"John":1, "likes":2, "to":3, "watch":4, "movies":5, "also":6, "football":7, "games":8, "Mary":9, "too":10}

词典包含10个单词，每个单词有唯一的索引。

开始的时候，使用one-hot方式表示每个词，即建立一个和词典一样大小的向量，然后该单词索引的位置用1，其他位置用0表示。
- john:[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
- likes:[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
- too:[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

这是一种离散方式的表示，但至少计算机能读懂了。

这样的方式有个问题，由于每个向量只有一个1，其余位置为0，并且互相的内积都是0。使得这些词之间没有关联互相独立。但实际情况肯定不是这样子（例如苹果和橘子的关联肯定比苹果和国王的关联要强），这使得算法对相关词的泛化能力不强。

换一种方式描述词，例如：Man, Woman, King, Queen, Apple, Orange，寻找一些特征，比如是否与性别有关，是否与高贵程度有关，是否与年龄有关，是否与食物有关，这样每个词都可能得到一种下面的表示方式：
<img style="float: center;" src="images/2.png" width="70%">

如果用这种方法来表示苹果和橘子，则苹果和橘子肯定会非常相似，至少大部分特征是一样的，对于已经知道橙子果汁的算法，很大几率会明白苹果果汁是什么东西。

对于不同的单词，算法会泛化地更好，并且，我们找的特征的个数一般会比词典小得多，比如找300个特征，则描述每个词的向量是300维，也比之前的one-hot的方式维度小得多。

这种捕捉到单词之间关联称为**词嵌入表示（Word-Embedding）**

如何得到这种表示？

首先需要一个嵌入矩阵Embedding Matrix，一开始用one-hot，即字典的位置表示每个词，然后通过嵌入矩阵，得到每个词的词嵌入向量：
<img style="float: center;" src="images/3.png" width="70%">

事先训练好一个词嵌入矩阵，该矩阵中每一列就是每个单词的词向量，每一行表示一个特征，上图300\*10000的矩阵，就是10000个单词，每个单词从300个特征上进行衡量。

有了这样一个矩阵之后，拿这个矩阵乘以每个单词自己的one-hot表示，就会得到每个单词的词向量表示。

词嵌入矩阵如何获取？
- 早先的时候，使用自然语言模型计算嵌入矩阵
- 例如：I want a glass of orange______
- 想让计算机填juice，嵌入矩阵未知，可以构建下面的神经网络进行训练：
<img style="float: center;" src="images/4.png" width="70%">

把嵌入矩阵也当作一层参数W，通过梯度下降的方式得到。

在训练网络的时候，不仅有orange juice，还有apple juice，在这个算法的激励下，苹果和橘子会学到很相似的嵌入，这样做能够让算法更好地符合训练集。因为它有时看到orange juice，有时看到apple juice，如果只有一个300维的特征向量来表示这些词，算法就会发现，要想更好地拟合数据集，就要使苹果，橘子，梨，葡萄等水果都具有相似的特征向量，这就是早期最成功的学习嵌入矩阵的算法之一。

但是如果单单为了得到嵌入矩阵而去训练一个模型会很复杂且耗时，于是人们想出一种简单的方式学习词嵌入（选上下文的方式），比如单纯只为了得到嵌入矩阵，根本没必要用一句话进行训练，选用几个单词对或者短语就可以，比如要预测juice，就可以把这个当作target，然后只考虑它周围的词就可以了（orange，a glass of orange等等这些就可以了）

一般通过某个单词周围的一些词就基本上海可以知道这个词的意思，比如单词bank，一般它周围的词都是money，government，finance等等，通过这些就可以推测bank与什么有关系了。这种上下文方式学习单词之间的关联，比起建立一个语言模型来说，要容易地多。

那么，对于一个句子，如何选择上下文和目标词呢？

可以用Skip-Gram模型，做法是**抽取上下文和目标词配对，来构造一个监督学习问题**。

这里的上下文不一定总是目标单词之前离得最近的4个单词或最近的n个单词，我们要做的是：
- 首先随机选择一个单词作为context，例如：orange
- 然后随机在一定距离内选另外一个词作为target（使用一个宽度为5或10的滑动窗口，在context附近选择一个单词作为target），可以是juice，glass，my等等。
- 最终得到多个【context-target对】作为监督学习样本
<img style="float: center;" src="images/1.jpg" width="70%">

skip-gram模型如何训练：
- 假设单词数为10000，随机选择上下文context c("orange")，然后根据滑动距离随机选择一个target t("juice")，让神经网络学习这个映射：
<img style="float: center;" src="images/2.jpg" width="70%">

训练的过程构建自然语言模型，经过softmax单元的输出为：$p(t|c)=\frac{e^{\theta^T_te_c}}{\sum^{10000}_{j=1}e^{\theta^T_je_c}}$

其中，$\theta_t$为target对应的参数，$e_c$为context的embedding vector，即$e_c=E\cdot o_c$

相应的loss函数为：$L(\hat{y},y)=-\sum^{10000}_{i=1}y_i\log\hat{y_i}$

之后使用梯度下降算法，迭代优化，最终得到嵌入矩阵E。

以上就是Skig-Gram模型：它把一个单词orange作为输入，并预测这个词从左数或者从右数的某个词，预测上下文词前面或后面的一些词是什么词。

简单理解：有一些正确的单词对，想让模型做一个训练，把上下文输入进去，预测出最终的目标。比如有orange-juice，orange-glass，orange-my，当输入orange时，分别输出后面的三个单词，这样训练好的时候，就可以通过中心词去预测周围的单词了。

Skip-Gram模型是Word2Vec的一种，Word2Vec另一种模型CBOW（Continuous Bag of Words），它获得中间词两边的上下文，去预测中间的词。

**缺点：**
- softmax公式中分母是求和，如果有10000个单词的时候，模型最后输出的时候都要考虑进来，10000个单词究竟哪个单词概率最大。（计算量过大）
- 论文中提到两个解决方法：
  - 层级softmax分类（Hierarchical softmax classifier）
  - 负采样方式

**负采样方式（Negative sample）**：判断选取的context和target是否构成一组正确的context-target对，一般包含一个正样本和k个负样本，例如：
- “orange”为context word，“juice”为target word，很明显“orange juice”是一组context-target对，为正样本，相应的target label为1。
- “orange”为context word不变，target word随机选择“king”、“book”、“the”或者“of”等。这些都是错误的context-target对，为负样本，相应的target label为0。

这就是如何生成训练集的方法，选一个正样本和k个负样本。

固定某个context word对应的负样本个数k遵循：
- 若训练样本较小，k一般选择5-20
- 若训练样本较大，k一般选择2-5即可

**从x映射到y的监督模型**
<img style="float: center;" src="images/5.png" width="70%">

负采样的数学模型：$P(y=1|c, t)=\sigma\left(\theta^T_t, e_c\right)$

其中，$\sigma$表示sigmoid激活函数，某个固定的正样本对应k个负样本（模型共包含k+1个二分类，对比之前的10000个输出单元的softmax分类，计算量小很多，大大提高模型运算速度）

每一次训练，都是k+1个二分类问题，就看target的那几个是不是我们想要的0或者1，然后用这几个去计算损失更新参数即可。

如何选择负样本对应的target单词？可以随机选择，但论文中提出一个更实用、效果更好的方法，就是根据该词出现的频率进行选择，概率公式为：$P(w_i)=\frac{f(w_i)^{3/4}}{\sum^{10000}_jf(w_j)^{3/4}}$

论文中损失函数：$loss=\log\sigma\left(v_{w_O}^{'T}v_{w_I}\right)+\sum^k_{i=1}\mathbb{E}_{w_i\sim P_n(w)}\left[\log\sigma(-v_{w_i}^{'T}v_{w_I})\right]$

损失函数理解：输入时选择的上下文，即此处的$v_{w_I}$，是embedding之后的向量，而输出是正负样本的embedding后的向量。

前面部分是正样本和上下文的关系，$v_{w_O}$就是正样本embedding后的形式，两个内积操作就是两者的关系程度（内积的几何意义，如果两个向量的关系越接近，则内积就会越大），后面那部分是负样本和上下文的关系，希望上下文与正样本的关系尽可能的近，也就是前面那部分越大越好，希望负样本与上下文的关系尽可能的小，但是后面发现内积前加了个负号，那就表示后面那部分越大越好。（最终损失函数越大越好，因此后面的损失函数要取相反数-loss）

# 实现skip-gram模型

思路：
- 建立一个词汇表（字典），根据训练集进行构建
- 根据这个词汇表，建立模型训练
- 训练过程中保存模型参数，测试的时候导入就可以直接进行预测

## 导入包

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud
from torch.nn.parameter import Parameter

from collections import Counter
import numpy as np
import pandas as pd
import random
import scipy
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

USE_CUDA = torch.cuda.is_available()

# 为了保证实验结果可以复现，经常把random seed固定在某一个值
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
if USE_CUDA:
    torch.cuda.manual_seed(1)

# 设置超参数
K = 100    # 负样本的个数, 每一个正样本对应100个负样本
C = 3    # 附近单词的门限
NUM_EPOCHS = 2   # 训练epoch数
MAX_VOCAB_SIZE = 30000   # 词典中单词的个数
LEARNING_RATE = 0.2   # 初始学习率
EMBEDDING_SIZE = 100   # 词向量特征的个数
BATCH_SIZE = 128

LOG_FILE = "word_embedding.log"

## 构建一张词汇表

从文本文件中读取所有的文字，然后通过这些文字创建一个vocabulary

In [2]:
# tokenize函数，把一篇文本转成一个个单词
def word_tokenize(text):
    return text.split()

with open("./data/text8.train.txt", 'r') as fin:
    text = fin.read()

# 把每句话的单词分开
text = [w for w in word_tokenize(text.lower())]

单词数量过大，选择最常见的30000个单词，后面不常用的统一用unk表示

In [3]:
vocab = dict(Counter(text).most_common(MAX_VOCAB_SIZE - 1))

添加一个UNK单词表示所有不常见的单词

In [4]:
vocab["<unk>"] = len(text) - np.sum(list(vocab.values()))

记录每个单词的index的mapping，以及index到单词的mapping，单词的count，单词frequency，以及单词总数

In [5]:
# 建立映射
idx_to_word = [word for word in vocab.keys()]
word_to_idx = {word: i for i, word in enumerate(idx_to_word)}

# 统计单词的频率和个数
word_counts = np.array([count for count in vocab.values()], dtype=np.float32)
word_freqs = word_counts / np.sum(word_counts)
word_freqs = word_freqs ** (3./4.)
word_freqs = word_freqs / np.sum(word_freqs)  # 论文中的计算频率的公式，选择中心词用
VOCAB_SIZE = len(idx_to_word)   # 30000个单词的词表建立完毕

## 实现DataLoader

DataLoader，是PyTorch中数据读取的一个重要接口，该接口定义在dataloader.py中，只要是用PyTorch来训练模型基本上都会用到这个接口。

目的：将自定义的Dataset根据batch size大小，是否shuffle等封装成一个batch size大小的Tensor，用于后面的训练。

有了Dataloader之后，就可以轻松随机打乱整个数据集，拿到一个batch size的数据。

注意：
- dataloader本质是一个可迭代对象，使用iter()访问，不能使用next()访问
- 使用iter(dataloader)返回一个迭代器，然后使用next访问
- 可以使用`for inputs, labels in dataloaders`进行可迭代对象的访问
- 同时需要实现dataset对象，传入到dataloader中，然后内部使用yeild返回每一次 batch的数据

一个比较好的写Dataloader教程：https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

torch.utils.data.Dataset表示数据集的抽象类，自定义数据集应继承Dataset并覆盖以下方法：
- \_\_len\_\_：函数需要返回整个数据集中有多少个item，之后通过len(dataset)返回数据集大小
- \_\_getitem\_\_：根据给定的index返回一个item，dataset[i]可以用于获取第i个样本

当前任务中，dataloader需要获取以下内容：
- 所有text编码成数字，然后用二次采样处理这些数字
- 保存字典表，单词数，词频
- 每个iteration sample一个中心词
- 根据当前中心词返回context单词
- 根据中心词采样一些负样本单词
- 返回单词的counts

In [6]:
# 首先，DataLoader继承torch.utils.data.Dataset
class WordEmbeddingDataset(tud.Dataset):
    # 把上面有的先保存下来
    def __init__(self, text, word_to_idx, idx_to_word, word_freqs, word_counts):
        ''' 
        text: a list of words, all text from the training dataset
        word_to_idx: the dictionary from word to idx
        idx_to_word: idx to word mapping
        word_freqs: the frequency of each word
        word_counts: the word counts
        '''
        super(WordEmbeddingDataset, self).__init__()
        # 训练集的每个单词在词典中的位置
        self.text_encode = [word_to_idx.get(word, VOCAB_SIZE - 1) for word in text]
        # 转成张量
        self.text_encode = torch.Tensor(self.text_encode).long()
        self.word_to_idx = word_to_idx
        self.idx_to_word = idx_to_word
        self.word_freqs = word_freqs
        self.word_counts = torch.Tensor(word_counts)

    # 返回整个数据集（所有单词的长度）
    def __len__(self):
        return len(self.text_encode)

    # 实现getitem函数  这个告诉模型应该怎么取数据，这是关键
    def __getitem__(self, idx):
        ''' 这个function返回以下数据用于训练
        - 中心词
        - 这个单词附近的(positive)单词
        - 随机采样的K个单词作为negative sample
        '''
        center_word = self.text_encode[idx]  # 中心词的位置
        # Windows的index
        pos_indics = list(range(idx - C,idx)) + list(range(idx + 1, idx + C + 1))
        # 取余， 防止超出text的长度
        pos_indics = [i % len(self.text_encode) for i in pos_indics]
        pos_words = self.text_encode[pos_indics]  # 正样本取周围单词
         # 根据单词的频率采样，对于每一个正确的单词，要采集K个错误的单词
        neg_words = torch.multinomial(torch.tensor(self.word_freqs), K * pos_words.shape[0], True)

        return center_word, pos_words, neg_words

新建一个dataset和DataLoader

In [7]:
dataset = WordEmbeddingDataset(text, word_to_idx, idx_to_word, word_freqs, word_counts)
dataset.text_encode.size()   # 15304686个单词

# 有了dataset之后，就可以非常简单的用DataLoader变成一个DataLoader
# 这样可以非常轻松的产生batch， 并且可以shuffle
dataloader = tud.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

取Batch的时候就非常简单了

In [8]:
# for i, (center_work, pos_words, neg_words) in enumerate(dataloader):
#     print(center_work, pos_words, neg_words)

# 这一个就会得到BATCH_SIZE个数据样本，每个数据样本都是中心词，正样本和负样本的形式
next(iter(dataloader))  

[tensor([ 1769,    62,    11, 21115,  6716, 29999,  1045,   138,   460, 29999,
         29999, 29999,     1,    18,  9235,     2,  8312,     9, 13824, 29999,
         29999,  1714,    22,  1562,     0,    30,     0,     0,    22,   990,
           445,    33,  4390,  7219,     0,   255,  1217,     5,     1,     4,
            29,    20,     0,  2487,  1534,   991,     3,    18,    28,   141,
           193,  2800,     1,     9,   337,   127,   157,   149,   107,   236,
             6,    84,    43, 29828,    52,   403,   186,    83,  1265,   552,
             2,   595,  2798,    33,    12,  8462,    10,   127,  8872,  1514,
          6620,   217,     1,  3680,  5804,     5, 29999,   956, 19058, 21389,
           495,    39,    30,   349,   416,    11,     5,     1,     1,     6,
             1,   122,   211,  2719,   873,     0,    22, 24620,    22,    93,
             0,   535,    99,  1734,     3,  1269,  5909,     3,    15,  5069,
          1706, 17024, 29999,    28,   169,   129,  

## 定义PyTorch模型

In [9]:
class EmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embed_size):
        """初始化输入和输出的embedding"""
        super(EmbeddingModel, self).__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size

        # 初始化
        initrange = 0.5 / self.embed_size
        self.out_embed = nn.Embedding(self.vocab_size, self.embed_size, sparse=False)
        self.out_embed.weight.data.uniform_(-initrange, initrange)

        self.in_embed = nn.Embedding(self.vocab_size, self.embed_size, sparse=False)
        # 这是在范围直接均匀分布采样
        self.in_embed.weight.data.uniform_(-initrange, initrange)

    def forward(self, input_labels, pos_labels, neg_labels):
        """
        input_labels: 中心词, [batch_size]
        pos_labels: 中心词周围 context window 出现过的单词 [batch_size * (window_size * 2)]
        neg_labels: 中心词周围没有出现过的单词，从 negative sampling 得到 [batch_size, (window_size * 2 * K)]

        return: loss, [batch_size]
        """
        
        batch_size = input_labels.size(0)

        input_embedding = self.in_embed(input_labels) # batchsize * embed_size
        pos_embedding = self.out_embed(pos_labels)   # batchsize*(2*c)*embed_size
        neg_embedding = self.out_embed(neg_labels) # batch_size * (2*C*K) * embed_size

        # 计算损失
        input_embedding = input_embedding.unsqueeze(2)  # [batchsize,embed_size, 1]
        log_pos = torch.bmm(pos_embedding, input_embedding).squeeze()   # [batchsize, 2*C]
        log_neg = torch.bmm(neg_embedding, -input_embedding).squeeze()   # [batchsize, 2*C*100]

        log_pos = F.logsigmoid(log_pos).sum(1)
        log_neg = F.logsigmoid(log_neg).sum(1)

        loss = log_pos + log_neg

        return -loss

    def input_embeddings(self):
        return self.in_embed.weight.data.cpu().numpy()

定义PyTorch模型，移动到GPU

In [10]:
model = EmbeddingModel(VOCAB_SIZE, EMBEDDING_SIZE)
if USE_CUDA:
    model.to('cuda')

## 训练模型

前向传播，计算损失，梯度清零，反向传播，参数更新

- 模型一般需要训练若干个epoch
- 每个epoch需要把所有数据分成若干个batch
- 每个batch的输入和输出都包装成cuda tensor
- 前向传播时，通过输入的句子预测每个单词的下一个单词
- 用预测的模型和正确的下一个单词计算交叉熵损失
- 清空模型当前的梯度
- 反向传播
- 更新模型参数
- 每隔一定的iteration输出模型在当前iteration的loss，并且保存参数

In [11]:
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
for e in range(NUM_EPOCHS):
    # 前面看看取batch是多么的方便，一句话就可以搞定
    for i, (input_labels, pos_labels, neg_labels) in enumerate(dataloader):

        # 先保证都是longTensor
        input_labels = input_labels.long()
        pos_labels = pos_labels.long()
        neg_labels = neg_labels.long()
        
        if USE_CUDA:
            input_labels = input_labels.cuda()
            pos_labels = pos_labels.cuda()
            neg_labels = neg_labels.cuda()
        
        optimizer.zero_grad()
        loss = model(input_labels, pos_labels, neg_labels).mean()
        loss.backward()
        optimizer.step()
        
        # 打印损失
        if i % 100 == 0:
            with open(LOG_FILE, "a") as fout:
                fout.write("epoch: {}, iter: {}, loss: {}\n".format(e, i, loss.item()))
                print("epoch: {}, iter: {}, loss: {}".format(e, i, loss.item()))

    # 保存参数
    embedding_weights = model.input_embeddings()
    np.save("embedding-{}".format(EMBEDDING_SIZE), embedding_weights)
    torch.save(model.state_dict(), "embedding-{}.th".format(EMBEDDING_SIZE))

epoch: 0, iter: 0, loss: 420.0471496582031
epoch: 0, iter: 100, loss: 293.7235107421875
epoch: 0, iter: 200, loss: 218.88555908203125
epoch: 0, iter: 300, loss: 173.64022827148438
epoch: 0, iter: 400, loss: 144.0834503173828
epoch: 0, iter: 500, loss: 141.6793212890625
epoch: 0, iter: 600, loss: 117.61723327636719
epoch: 0, iter: 700, loss: 135.5240020751953
epoch: 0, iter: 800, loss: 98.03471374511719
epoch: 0, iter: 900, loss: 96.02230072021484
epoch: 0, iter: 1000, loss: 86.41474151611328
epoch: 0, iter: 1100, loss: 92.35974884033203
epoch: 0, iter: 1200, loss: 87.557373046875
epoch: 0, iter: 1300, loss: 75.23333740234375
epoch: 0, iter: 1400, loss: 85.7183609008789
epoch: 0, iter: 1500, loss: 75.12042236328125
epoch: 0, iter: 1600, loss: 69.20245361328125
epoch: 0, iter: 1700, loss: 70.73925018310547
epoch: 0, iter: 1800, loss: 71.04591369628906
epoch: 0, iter: 1900, loss: 69.52195739746094
epoch: 0, iter: 2000, loss: 65.73233032226562
epoch: 0, iter: 2100, loss: 69.12045288085938


epoch: 0, iter: 17600, loss: 33.021976470947266
epoch: 0, iter: 17700, loss: 32.39189147949219
epoch: 0, iter: 17800, loss: 32.150604248046875
epoch: 0, iter: 17900, loss: 32.82508087158203
epoch: 0, iter: 18000, loss: 32.668113708496094
epoch: 0, iter: 18100, loss: 32.69425582885742
epoch: 0, iter: 18200, loss: 32.16299057006836
epoch: 0, iter: 18300, loss: 32.27485656738281
epoch: 0, iter: 18400, loss: 32.31639099121094
epoch: 0, iter: 18500, loss: 31.150318145751953
epoch: 0, iter: 18600, loss: 31.87506103515625
epoch: 0, iter: 18700, loss: 32.12548065185547
epoch: 0, iter: 18800, loss: 32.14763259887695
epoch: 0, iter: 18900, loss: 32.636192321777344
epoch: 0, iter: 19000, loss: 31.977798461914062
epoch: 0, iter: 19100, loss: 32.861995697021484
epoch: 0, iter: 19200, loss: 32.619117736816406
epoch: 0, iter: 19300, loss: 32.857994079589844
epoch: 0, iter: 19400, loss: 32.28169250488281
epoch: 0, iter: 19500, loss: 32.14692306518555
epoch: 0, iter: 19600, loss: 32.80815124511719
epoc

epoch: 0, iter: 34900, loss: 30.85108184814453
epoch: 0, iter: 35000, loss: 31.109275817871094
epoch: 0, iter: 35100, loss: 31.504987716674805
epoch: 0, iter: 35200, loss: 31.164974212646484
epoch: 0, iter: 35300, loss: 31.425212860107422
epoch: 0, iter: 35400, loss: 31.558650970458984
epoch: 0, iter: 35500, loss: 31.296070098876953
epoch: 0, iter: 35600, loss: 31.726829528808594
epoch: 0, iter: 35700, loss: 31.459388732910156
epoch: 0, iter: 35800, loss: 31.363842010498047
epoch: 0, iter: 35900, loss: 31.478790283203125
epoch: 0, iter: 36000, loss: 31.512521743774414
epoch: 0, iter: 36100, loss: 31.50742530822754
epoch: 0, iter: 36200, loss: 31.437950134277344
epoch: 0, iter: 36300, loss: 32.16291046142578
epoch: 0, iter: 36400, loss: 31.010114669799805
epoch: 0, iter: 36500, loss: 31.171680450439453
epoch: 0, iter: 36600, loss: 31.108970642089844
epoch: 0, iter: 36700, loss: 31.33449935913086
epoch: 0, iter: 36800, loss: 31.325313568115234
epoch: 0, iter: 36900, loss: 30.625179290771

epoch: 0, iter: 52100, loss: 31.361873626708984
epoch: 0, iter: 52200, loss: 30.893741607666016
epoch: 0, iter: 52300, loss: 30.718006134033203
epoch: 0, iter: 52400, loss: 31.015762329101562
epoch: 0, iter: 52500, loss: 31.00485610961914
epoch: 0, iter: 52600, loss: 31.05432891845703
epoch: 0, iter: 52700, loss: 30.820741653442383
epoch: 0, iter: 52800, loss: 30.87388801574707
epoch: 0, iter: 52900, loss: 31.245819091796875
epoch: 0, iter: 53000, loss: 31.113101959228516
epoch: 0, iter: 53100, loss: 31.04051971435547
epoch: 0, iter: 53200, loss: 30.783733367919922
epoch: 0, iter: 53300, loss: 30.71259307861328
epoch: 0, iter: 53400, loss: 31.52549934387207
epoch: 0, iter: 53500, loss: 31.074138641357422
epoch: 0, iter: 53600, loss: 30.682376861572266
epoch: 0, iter: 53700, loss: 31.03972625732422
epoch: 0, iter: 53800, loss: 30.844356536865234
epoch: 0, iter: 53900, loss: 31.168516159057617
epoch: 0, iter: 54000, loss: 30.87356948852539
epoch: 0, iter: 54100, loss: 31.002277374267578


epoch: 0, iter: 69400, loss: 30.816993713378906
epoch: 0, iter: 69500, loss: 30.307523727416992
epoch: 0, iter: 69600, loss: 30.94533920288086
epoch: 0, iter: 69700, loss: 30.840808868408203
epoch: 0, iter: 69800, loss: 30.76001739501953
epoch: 0, iter: 69900, loss: 30.81319808959961
epoch: 0, iter: 70000, loss: 30.45636749267578
epoch: 0, iter: 70100, loss: 30.786243438720703
epoch: 0, iter: 70200, loss: 30.836938858032227
epoch: 0, iter: 70300, loss: 30.81452178955078
epoch: 0, iter: 70400, loss: 30.829416275024414
epoch: 0, iter: 70500, loss: 30.64291763305664
epoch: 0, iter: 70600, loss: 31.78078269958496
epoch: 0, iter: 70700, loss: 30.614402770996094
epoch: 0, iter: 70800, loss: 31.274394989013672
epoch: 0, iter: 70900, loss: 30.765005111694336
epoch: 0, iter: 71000, loss: 30.989028930664062
epoch: 0, iter: 71100, loss: 31.00112533569336
epoch: 0, iter: 71200, loss: 30.464740753173828
epoch: 0, iter: 71300, loss: 30.848350524902344
epoch: 0, iter: 71400, loss: 31.216331481933594


...

epoch: 1, iter: 35900, loss: 30.534183502197266
epoch: 1, iter: 36000, loss: 30.009206771850586
epoch: 1, iter: 36100, loss: 30.621532440185547
epoch: 1, iter: 36200, loss: 30.682458877563477
epoch: 1, iter: 36300, loss: 30.842975616455078
epoch: 1, iter: 36400, loss: 30.65481185913086
epoch: 1, iter: 36500, loss: 30.222368240356445
epoch: 1, iter: 36600, loss: 30.546770095825195
epoch: 1, iter: 36700, loss: 30.72589874267578
epoch: 1, iter: 36800, loss: 30.210983276367188
epoch: 1, iter: 36900, loss: 30.662126541137695
epoch: 1, iter: 37000, loss: 30.339113235473633
epoch: 1, iter: 37100, loss: 30.632240295410156
epoch: 1, iter: 37200, loss: 30.11248779296875
epoch: 1, iter: 37300, loss: 30.81224822998047
epoch: 1, iter: 37400, loss: 29.901453018188477
epoch: 1, iter: 37500, loss: 30.180368423461914
epoch: 1, iter: 37600, loss: 30.85427474975586
epoch: 1, iter: 37700, loss: 31.092266082763672
epoch: 1, iter: 37800, loss: 30.42523956298828
epoch: 1, iter: 37900, loss: 30.02549743652343

epoch: 1, iter: 53200, loss: 30.619110107421875
epoch: 1, iter: 53300, loss: 30.695589065551758
epoch: 1, iter: 53400, loss: 30.62221908569336
epoch: 1, iter: 53500, loss: 30.03057861328125
epoch: 1, iter: 53600, loss: 30.39617156982422
epoch: 1, iter: 53700, loss: 30.148765563964844
epoch: 1, iter: 53800, loss: 30.24462127685547
epoch: 1, iter: 53900, loss: 30.461177825927734
epoch: 1, iter: 54000, loss: 30.368053436279297
epoch: 1, iter: 54100, loss: 30.626056671142578
epoch: 1, iter: 54200, loss: 30.74512481689453
epoch: 1, iter: 54300, loss: 30.458711624145508
epoch: 1, iter: 54400, loss: 29.78676414489746
epoch: 1, iter: 54500, loss: 30.59987449645996
epoch: 1, iter: 54600, loss: 30.543113708496094
epoch: 1, iter: 54700, loss: 30.33462905883789
epoch: 1, iter: 54800, loss: 30.524120330810547
epoch: 1, iter: 54900, loss: 30.575603485107422
epoch: 1, iter: 55000, loss: 30.370426177978516
epoch: 1, iter: 55100, loss: 30.73145294189453
epoch: 1, iter: 55200, loss: 30.255538940429688
e

epoch: 1, iter: 70400, loss: 30.96818733215332
epoch: 1, iter: 70500, loss: 30.504425048828125
epoch: 1, iter: 70600, loss: 30.785526275634766
epoch: 1, iter: 70700, loss: 30.771575927734375
epoch: 1, iter: 70800, loss: 30.684459686279297
epoch: 1, iter: 70900, loss: 30.338008880615234
epoch: 1, iter: 71000, loss: 30.76706886291504
epoch: 1, iter: 71100, loss: 30.35271453857422
epoch: 1, iter: 71200, loss: 29.788108825683594
epoch: 1, iter: 71300, loss: 30.233728408813477
epoch: 1, iter: 71400, loss: 30.482433319091797
epoch: 1, iter: 71500, loss: 30.675006866455078
epoch: 1, iter: 71600, loss: 30.30660057067871
epoch: 1, iter: 71700, loss: 30.658329010009766
epoch: 1, iter: 71800, loss: 29.85631561279297
epoch: 1, iter: 71900, loss: 30.80324935913086
epoch: 1, iter: 72000, loss: 30.16228675842285
epoch: 1, iter: 72100, loss: 30.215011596679688
epoch: 1, iter: 72200, loss: 30.666141510009766
epoch: 1, iter: 72300, loss: 30.553688049316406
epoch: 1, iter: 72400, loss: 30.20686912536621


epoch: 1, iter: 87700, loss: 30.229537963867188
epoch: 1, iter: 87800, loss: 30.521480560302734
epoch: 1, iter: 87900, loss: 29.76840591430664
epoch: 1, iter: 88000, loss: 30.632104873657227
epoch: 1, iter: 88100, loss: 30.671722412109375
epoch: 1, iter: 88200, loss: 30.387523651123047
epoch: 1, iter: 88300, loss: 30.784847259521484
epoch: 1, iter: 88400, loss: 30.54539680480957
epoch: 1, iter: 88500, loss: 30.189746856689453
epoch: 1, iter: 88600, loss: 30.59088897705078
epoch: 1, iter: 88700, loss: 29.246814727783203
epoch: 1, iter: 88800, loss: 30.34463119506836
epoch: 1, iter: 88900, loss: 30.282146453857422
epoch: 1, iter: 89000, loss: 30.648700714111328
epoch: 1, iter: 89100, loss: 29.976829528808594
epoch: 1, iter: 89200, loss: 30.621397018432617
epoch: 1, iter: 89300, loss: 30.41439437866211
epoch: 1, iter: 89400, loss: 30.325531005859375
epoch: 1, iter: 89500, loss: 30.507482528686523
epoch: 1, iter: 89600, loss: 30.216299057006836
epoch: 1, iter: 89700, loss: 30.3153858184814

epoch: 1, iter: 104800, loss: 30.385835647583008
epoch: 1, iter: 104900, loss: 30.136550903320312
epoch: 1, iter: 105000, loss: 29.920372009277344
epoch: 1, iter: 105100, loss: 30.32604217529297
epoch: 1, iter: 105200, loss: 30.486291885375977
epoch: 1, iter: 105300, loss: 30.22525405883789
epoch: 1, iter: 105400, loss: 30.65369415283203
epoch: 1, iter: 105500, loss: 30.271739959716797
epoch: 1, iter: 105600, loss: 30.71205711364746
epoch: 1, iter: 105700, loss: 30.560075759887695
epoch: 1, iter: 105800, loss: 30.604328155517578
epoch: 1, iter: 105900, loss: 30.546695709228516
epoch: 1, iter: 106000, loss: 30.05081558227539
epoch: 1, iter: 106100, loss: 30.21908187866211
epoch: 1, iter: 106200, loss: 30.654855728149414
epoch: 1, iter: 106300, loss: 30.81117057800293
epoch: 1, iter: 106400, loss: 30.75257110595703
epoch: 1, iter: 106500, loss: 30.158058166503906
epoch: 1, iter: 106600, loss: 30.74936294555664
epoch: 1, iter: 106700, loss: 30.405668258666992
epoch: 1, iter: 106800, loss:

In [12]:
# 之后需要再次使用嵌入矩阵时，可以直接导入
model.load_state_dict(torch.load("embedding-{}.th".format(EMBEDDING_SIZE)))

# 我们要的是这个权重
embedding_weights = model.input_embeddings()

## 模型的测试

### 寻找最近邻

In [13]:
def find_nearest(word):
    index = word_to_idx[word]
    embedding = embedding_weights[index]
    cos_dis = np.array([scipy.spatial.distance.cosine(e, embedding) for e in embedding_weights])
    return [idx_to_word[i] for i in cos_dis.argsort()[:10]]

# 找和下面几个单词相近的单词：
for word in ["good", "fresh", "monster", "green", "like", "america", "chicago", "work", "computer", "language"]:
    print(word, find_nearest(word))

good ['good', 'bad', 'perfect', 'hard', 'truth', 'alone', 'really', 'money', 'heart', 'doing']
fresh ['fresh', 'grain', 'waste', 'sized', 'lighter', 'minimal', 'noise', 'clean', 'fiber', 'cooling']
monster ['monster', 'giant', 'robot', 'clown', 'snake', 'demon', 'bird', 'hammer', 'triangle', 'rod']
green ['green', 'blue', 'yellow', 'white', 'cross', 'orange', 'red', 'black', 'mountain', 'snow']
like ['like', 'etc', 'unlike', 'similarly', 'soft', 'fish', 'rich', 'eat', 'whereas', 'sounds']
america ['america', 'korea', 'africa', 'india', 'australia', 'turkey', 'pakistan', 'argentina', 'europe', 'asia']
chicago ['chicago', 'boston', 'texas', 'illinois', 'massachusetts', 'london', 'florida', 'berkeley', 'toronto', 'indiana']
work ['work', 'writing', 'writings', 'marx', 'speech', 'vision', 'philosophical', 'job', 'appearance', 'genre']
computer ['computer', 'digital', 'software', 'electronic', 'graphics', 'video', 'audio', 'hardware', 'computers', 'program']
language ['language', 'alphabet'

可以发现与good类似的有bad，perfect，与green有关的有blue，yellow，white这些颜色，与America有关的都是一些国家的一些词，效果不错

### 单词之间关系的类比推理

词嵌入有一个很好的特性，就是它能帮助实现类比推理。

加入提出一个问题，男人对应女人，则king对应什么？能否有一种算法可以自动推导出这种关系。
<img style="float: center;" src="images/6.png" width="70%">

In [14]:
man_idx = word_to_idx["man"] 
king_idx = word_to_idx["king"] 
woman_idx = word_to_idx["woman"]
embedding = embedding_weights[woman_idx] - embedding_weights[man_idx] + embedding_weights[king_idx]
cos_dis = np.array([scipy.spatial.distance.cosine(e, embedding) for e in embedding_weights])
for i in cos_dis.argsort()[:20]:
    print(idx_to_word[i])

king
henry
charles
pope
queen
iii
edward
elizabeth
prince
alexander
iv
constantine
james
frederick
louis
joseph
albert
mary
sir
vii


可以看到国王对应上面的这些，里面也有queen，伊丽莎白等